In [3]:
import xml.etree.ElementTree as ET

class Gate:
    def __init__(self,id,type):
        self.inputs = []
        self.id = id
        self.type = type


class InputGate(Gate):
    def __init__(self, value,id,type):
        super().__init__(id,type)
        self.value = value

    def output(self):
        return self.value

class AndGate(Gate):
    def __init__(self, id,type):
        super().__init__(id,type)
    def output(self):
        return all(input_gate.output() for input_gate in self.inputs)

class OrGate(Gate):
    def __init__(self, id,type):
        super().__init__(id,type)
    def output(self):
        return any(input_gate.output() for input_gate in self.inputs)
class NandGate(Gate):
    def __init__(self, id,type):
        super().__init__(id,type)
    def output(self):
        return not all(input_gate.output() for input_gate in self.inputs)
class NorGate(Gate):
    def __init__(self, id,type):
        super().__init__(id,type)
    def output(self):
        return not any(input_gate.output() for input_gate in self.inputs)
class XnorGate(Gate):
    def __init__(self, id,type):
        super().__init__(id,type)
    def output(self):
        return not sum(input_gate.output() for input_gate in self.inputs) % 2
class XorGate(Gate):
    def __init__(self, id,type):
        super().__init__(id,type)
    def output(self):
        return sum(input_gate.output() for input_gate in self.inputs) % 2

class NotGate(Gate):
    def __init__(self, id,type):
        super().__init__(id,type)
    def output(self):
        return not self.inputs[0].output()
    
def build_circuit(xml_data):
    tree = ET.ElementTree(ET.fromstring(xml_data))
    gates = []
    outputs = {}
    index=0
    for gate in tree.findall('gate'):
        if gate.get('type') == 'IN':
            gates.append(InputGate(value=False,id=index,type='IN')) 
        elif gate.get('type') == 'AND':
            gates.append(AndGate(id=index,type='AND'))
        elif gate.get('type') == 'NAND':
            gates.append(NandGate(id=index,type='NAND'))
        elif gate.get('type') == 'OR':
            gates.append(OrGate(id=index,type='OR'))
        elif gate.get('type') == 'NOR':
            gates.append(NorGate(id=index,type='NOR'))
        elif gate.get('type') == 'XOR':
            gates.append(XorGate(id=index,type='XOR'))
        elif gate.get('type') == 'XNOR':
            gates.append(XnorGate(id=index,type='XNOR'))
        elif gate.get('type') == 'NOT':
            gates.append(NotGate(id=index,type='NOT'))
        index+=1

    for gate in tree.findall('gate'):
        gate_index = tree.findall('gate').index(gate)
        for input_gate in gate.findall('input'):
            input_index = int(input_gate.get('gate'))
            gates[gate_index].inputs.append(gates[input_index])

    for output in tree.findall('output'):
        bit = int(output.get('bit'))
        gate_index = int(output.get('gate'))
        outputs[bit] = gates[gate_index]

    return gates, outputs



In [13]:
def calculate_depth(gate, gate_depths):
    # Base case for input gates
    if isinstance(gate, InputGate) or gate in gate_depths:
        return gate_depths.get(gate, 0)

    # Calculate the depth of each input gate
    input_depths = [calculate_depth(input_gate, gate_depths) for input_gate in gate.inputs]

    # The depth of the current gate is 1 more than the maximum depth of its inputs
    current_depth = 1 + max(input_depths, default=0)
    gate_depths[gate] = current_depth
    return current_depth

def find_circuit_depth(gates, outputs):
    gate_depths = {}
    max_depth = 0
    for output in outputs.values():
        depth = calculate_depth(output, gate_depths)
        if depth > max_depth:
            max_depth = depth
    return max_depth


In [4]:
def calculate_circuit_cost(gates):
    # Define the costs for each gate type for each technology excluding Software and Multiplicative.
    tech_costs = {
        'UMC 180nm': {'AND': 1.33, 'OR': 1.33, 'NOT': 0.67, 'XOR': 3.00, 'XNOR': 3.00, 'NAND': 1, 'NOR': 1},
        'TSMC 65nm': {'AND': 1.50, 'OR': 1.50, 'NOT': 0.50, 'XOR': 3.00, 'XNOR': 3.00, 'NAND': 1, 'NOR': 1},
        'Depth (GEs)': {'AND': 1.50, 'OR': 1.50, 'NOT': 0.50, 'XOR': 2.00, 'XNOR': 2.00, 'NAND': 1, 'NOR': 1},
        'Depth (Soft.)': {'AND': 1.00, 'OR': 1.00, 'NOT': 1.00, 'XOR': 1.00, 'XNOR': 1.00, 'NAND': 1, 'NOR': 1},
    }

    # Initialize the cost dictionary
    total_costs = {tech: 0 for tech in tech_costs}

    # Count the number of each type of gate
    gate_counts = {'AND': 0, 'OR': 0, 'NOT': 0, 'XOR': 0, 'XNOR': 0, 'NAND': 0, 'NOR': 0}
    for gate in gates:
        if isinstance(gate, AndGate):
            gate_counts['AND'] += 1
        elif isinstance(gate, OrGate):
            gate_counts['OR'] += 1
        elif isinstance(gate, NotGate):
            gate_counts['NOT'] += 1
        elif isinstance(gate, XorGate):
            gate_counts['XOR'] += 1
        elif isinstance(gate, NandGate):
            gate_counts['NAND'] += 1
        elif isinstance(gate, NorGate):
            gate_counts['NOR'] += 1
        elif isinstance(gate, XnorGate):
            gate_counts['XNOR'] += 1
    print(gate_counts)
        

    # Calculate the total cost for each technology
    for tech, costs in tech_costs.items():
        for gate_type, count in gate_counts.items():
            total_costs[tech] += costs[gate_type] * count

    return total_costs

In [24]:
lut = ["4", "b", "1f", "14", "1a", "15", "9", "2", "1b", "5", "8", "12", "1d", "3", "6", "1c", "1e", "13", "7", "e", "0", "d", "11", "18", "10", "c", "1", "19", "16", "a", "f", "17"]

def write_lut_to_file(lut):
    with open('lut.txt', 'w') as f:
        for item in lut:
            f.write("%s " % item)
write_lut_to_file(lut)

In [25]:
gates = ["FALSE", "AND", "A AND NOT B", "A", "NOT A AND B", "B", "XOR", "OR", 
         "NOR", "XNOR", "NOT B", "A OR NOT B", "NOT A", "NOT A OR B", "NAND", "TRUE"]
values = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768]

gate_dict = {gate: value for gate, value in zip(gates, values)}

In [26]:
avaliable_gates="17346" # 1,7,3,4,6
#avaliable_gates= gate_dict['AND'] + gate_dict['OR']  + gate_dict['NOR']+ gate_dict['NAND']

In [34]:
import os
os.system("rm -r /Users/mehmetalidemir/web-docker/app/sboxgen/build/*xml")
os.system("pwd")
os.system("/Users/mehmetalidemir/web-docker/app/sboxgen/build/sboxgates ./sboxes/ascon.txt -a 17346")

/Users/mehmetalidemir/web-docker/app/sboxgen


rm: /Users/mehmetalidemir/web-docker/app/sboxgen/build/*xml: No such file or directory
sh: /Users/mehmetalidemir/web-docker/app/sboxgen/build/sboxgates: cannot execute binary file


32256

In [39]:
xml_file_name = ""
for file in os.listdir("/Users/mehmetalidemir/web-docker/app/sboxgen/build"):
    if file.endswith(".xml") and file.startswith("5"):
        xml_file_name = file
        break

In [40]:
with open(xml_file_name, 'r') as xml_file:
    xml_data = xml_file.read()
gates, outputs = build_circuit(xml_data)

FileNotFoundError: [Errno 2] No such file or directory: '5-023-0220-02431-c1b5725d.xml'

In [10]:
def create_cirtuit_and_calculate_cost():
    os.system("rm -r /Users/mehmetalidemir/web-docker/app/sboxgen/build/*xml")
    os.system("/Users/mehmetalidemir/web-docker/app/sboxgen/build/sboxgates /Users/mehmetalidemir/web-docker/app/sboxgen/sboxes/ascon.txt -a 17346")
    xml_file_name = ""
    for file in os.listdir("/Users/mehmetalidemir/web-docker/app/sboxgen/build/"):
        if file.endswith(".xml") and file.startswith("5"):
            xml_file_name = file
            break
    with open(xml_file_name, 'r') as xml_file:
        xml_data = xml_file.read()
    gates, outputs = build_circuit(xml_data)
    total_cost = calculate_circuit_cost(gates)
    depth = find_circuit_depth(gates, outputs)
    return total_cost,depth,gates,outputs

In [29]:
for i in range(100):
    t,d,gates,outputs = create_cirtuit_and_calculate_cost()
    print(t,d)
    if d == 5:
        break

{'AND': 2, 'OR': 3, 'NOT': 1, 'XOR': 8, 'XNOR': 5, 'NAND': 1, 'NOR': 2}
{'UMC 180nm': 49.32, 'TSMC 65nm': 50.0, 'Depth (GEs)': 37.0, 'Depth (Soft.)': 22.0} 9
{'AND': 2, 'OR': 3, 'NOT': 0, 'XOR': 10, 'XNOR': 4, 'NAND': 2, 'NOR': 0}
{'UMC 180nm': 50.65, 'TSMC 65nm': 51.5, 'Depth (GEs)': 37.5, 'Depth (Soft.)': 21.0} 9
{'AND': 3, 'OR': 2, 'NOT': 1, 'XOR': 7, 'XNOR': 6, 'NAND': 1, 'NOR': 1}
{'UMC 180nm': 48.32, 'TSMC 65nm': 49.0, 'Depth (GEs)': 36.0, 'Depth (Soft.)': 21.0} 8
{'AND': 1, 'OR': 3, 'NOT': 0, 'XOR': 8, 'XNOR': 5, 'NAND': 1, 'NOR': 1}
{'UMC 180nm': 46.32, 'TSMC 65nm': 47.0, 'Depth (GEs)': 34.0, 'Depth (Soft.)': 19.0} 6
{'AND': 3, 'OR': 2, 'NOT': 0, 'XOR': 9, 'XNOR': 3, 'NAND': 3, 'NOR': 1}
{'UMC 180nm': 46.65, 'TSMC 65nm': 47.5, 'Depth (GEs)': 35.5, 'Depth (Soft.)': 21.0} 6
{'AND': 1, 'OR': 3, 'NOT': 1, 'XOR': 9, 'XNOR': 4, 'NAND': 2, 'NOR': 1}
{'UMC 180nm': 47.99, 'TSMC 65nm': 48.5, 'Depth (GEs)': 35.5, 'Depth (Soft.)': 21.0} 9
{'AND': 1, 'OR': 3, 'NOT': 0, 'XOR': 8, 'XNOR': 5, 

In [25]:
def generate_binary_combinations(n):
    """n bitlik tüm ikili kombinasyonları üretir."""
    return [bin(i)[2:].zfill(n) for i in range(2**n)]


In [26]:
""" for gate in gates:
    print("gate: ",gate.type," id:",gate.id)
    for input_gate in gate.inputs:

        print("inputs : ",input_gate.id)

for output in outputs:
    print("output: ",output," id:",outputs[output].id) """

' for gate in gates:\n    print("gate: ",gate.type," id:",gate.id)\n    for input_gate in gate.inputs:\n\n        print("inputs : ",input_gate.id)\n\nfor output in outputs:\n    print("output: ",output," id:",outputs[output].id) '

In [27]:
def create_verilog(gates, outputs):
    verilog_str = "module circuit(\n"
    input_ids = [gate.id for gate in gates if isinstance(gate, InputGate)]
    input_str = "input " + ", ".join([f"in{input_id}" for input_id in input_ids]) + ",\n"
    output_str = "output " + ", ".join([f"out{bit}" for bit in outputs]) + "\n);\n\n"

    wire_str = ""
    gate_str = ""

    for gate in gates:
        if not isinstance(gate, InputGate):
            gate_output = f"g{gate.id}"
            wire_str += f"    wire {gate_output};\n"
            gate_inputs = ", ".join([f"in{inp.id}" if isinstance(inp, InputGate) else f"g{inp.id}" for inp in gate.inputs])

            if isinstance(gate, AndGate):
                gate_str += f"    and and{gate.id}({gate_output}, {gate_inputs});\n"
            elif isinstance(gate, OrGate):
                gate_str += f"    or or{gate.id}({gate_output}, {gate_inputs});\n"
            elif isinstance(gate, XorGate):
                gate_str += f"    xor xor{gate.id}({gate_output}, {gate_inputs});\n"
            elif isinstance(gate, NotGate):
                gate_str += f"    not not{gate.id}({gate_output}, {gate_inputs});\n"

    assign_str = "\n".join([f"    assign out{bit} = g{outputs[bit].id};" for bit in outputs]) + "\n"

    verilog_str += input_str + output_str + wire_str + gate_str + assign_str + "endmodule"

    return verilog_str

# Örnek kullanım
verilog_code = create_verilog(gates, outputs)
print(verilog_code)


module circuit(
input in0, in1, in2, in3, in4,
output out0, out1, out2, out3, out4
);

    wire g5;
    wire g6;
    wire g7;
    wire g8;
    wire g9;
    wire g10;
    wire g11;
    wire g12;
    wire g13;
    wire g14;
    wire g15;
    wire g16;
    wire g17;
    wire g18;
    wire g19;
    wire g20;
    wire g21;
    wire g22;
    wire g23;
    wire g24;
    or or7(g7, in3, in0);
    xor xor8(g8, g6, g7);
    xor xor9(g9, in1, in0);
    or or10(g10, g9, in4);
    xor xor11(g11, g10, in3);
    xor xor12(g12, g11, in2);
    xor xor14(g14, g13, in1);
    or or15(g15, in1, in0);
    xor xor16(g16, g14, g15);
    xor xor18(g18, g17, in4);
    or or19(g19, g18, in3);
    xor xor20(g20, g9, g19);
    and and23(g23, g16, in1);
    xor xor24(g24, g22, g23);
    assign out0 = g8;
    assign out1 = g12;
    assign out2 = g16;
    assign out3 = g24;
    assign out4 = g20;
endmodule


In [28]:
binary_combinations = generate_binary_combinations(5)

for combination in binary_combinations:
    for i, bit in enumerate(combination):
        gates[i].value = (bit == '1')

    output_values = [outputs[bit].output() for bit in sorted(outputs)]

    #print(f"Input: {combination[::-1]}, Output: {output_values[::-1]}")
    #write input and output hex format
    input_hex = hex(int(combination[::-1],2))
    output_hex = hex(int(''.join(str(i) for i in output_values[::-1]),2))
    print(f"Input Hex: {input_hex}, Output Hex: {output_hex}")

Input Hex: 0x0, Output Hex: 0x4
Input Hex: 0x10, Output Hex: 0x1e
Input Hex: 0x8, Output Hex: 0x1b
Input Hex: 0x18, Output Hex: 0x10
Input Hex: 0x4, Output Hex: 0x1a
Input Hex: 0x14, Output Hex: 0x0
Input Hex: 0xc, Output Hex: 0x1d
Input Hex: 0x1c, Output Hex: 0x16
Input Hex: 0x2, Output Hex: 0x1f
Input Hex: 0x12, Output Hex: 0x7
Input Hex: 0xa, Output Hex: 0x8
Input Hex: 0x1a, Output Hex: 0x1
Input Hex: 0x6, Output Hex: 0x9
Input Hex: 0x16, Output Hex: 0x11
Input Hex: 0xe, Output Hex: 0x6
Input Hex: 0x1e, Output Hex: 0xf
Input Hex: 0x1, Output Hex: 0xb
Input Hex: 0x11, Output Hex: 0x13
Input Hex: 0x9, Output Hex: 0x5
Input Hex: 0x19, Output Hex: 0xc
Input Hex: 0x5, Output Hex: 0x15
Input Hex: 0x15, Output Hex: 0xd
Input Hex: 0xd, Output Hex: 0x3
Input Hex: 0x1d, Output Hex: 0xa
Input Hex: 0x3, Output Hex: 0x14
Input Hex: 0x13, Output Hex: 0xe
Input Hex: 0xb, Output Hex: 0x12
Input Hex: 0x1b, Output Hex: 0x19
Input Hex: 0x7, Output Hex: 0x2
Input Hex: 0x17, Output Hex: 0x18
Input Hex: 

In [11]:
binary_combinations = generate_binary_combinations(5)

for combination in binary_combinations:
    for i, bit in enumerate(combination):
        gates[i].value = (bit == '1')

    output_values = [outputs[bit].output() for bit in sorted(outputs)]

    #print(f"Input: {combination[::-1]}, Output: {output_values[::-1]}")
    #write input and output hex format
    input_hex = hex(int(combination[::-1],2))
    output_hex = hex(int(''.join(str(i) for i in output_values[::-1]),2))
    print(f"Input Hex: {input_hex}, Output Hex: {output_hex}")

Input Hex: 0x0, Output Hex: 0x4
Input Hex: 0x10, Output Hex: 0x1e
Input Hex: 0x8, Output Hex: 0x1b
Input Hex: 0x18, Output Hex: 0x10
Input Hex: 0x4, Output Hex: 0x1a
Input Hex: 0x14, Output Hex: 0x0
Input Hex: 0xc, Output Hex: 0x1d
Input Hex: 0x1c, Output Hex: 0x16
Input Hex: 0x2, Output Hex: 0x1f
Input Hex: 0x12, Output Hex: 0x7
Input Hex: 0xa, Output Hex: 0x8
Input Hex: 0x1a, Output Hex: 0x1
Input Hex: 0x6, Output Hex: 0x9
Input Hex: 0x16, Output Hex: 0x11
Input Hex: 0xe, Output Hex: 0x6
Input Hex: 0x1e, Output Hex: 0xf
Input Hex: 0x1, Output Hex: 0xb
Input Hex: 0x11, Output Hex: 0x13
Input Hex: 0x9, Output Hex: 0x5
Input Hex: 0x19, Output Hex: 0xc
Input Hex: 0x5, Output Hex: 0x15
Input Hex: 0x15, Output Hex: 0xd
Input Hex: 0xd, Output Hex: 0x3
Input Hex: 0x1d, Output Hex: 0xa
Input Hex: 0x3, Output Hex: 0x14
Input Hex: 0x13, Output Hex: 0xe
Input Hex: 0xb, Output Hex: 0x12
Input Hex: 0x1b, Output Hex: 0x19
Input Hex: 0x7, Output Hex: 0x2
Input Hex: 0x17, Output Hex: 0x18
Input Hex: 